## 카카오 지오코딩 완성

In [4]:
import pandas as pd
import re
import requests, json

# 위경도 찾을 파일 불러오기
file_path = './data/고양시.csv'
goyang = pd.read_csv(file_path)

# 주소 전처리
pattern = r'\([^)]*\)'
text = []

for i in range(0, len(goyang['주소']),1):
    text.append(re.sub(pattern=pattern, repl='', string= goyang['주소'][i]))

text
goyang['주소'] = text

# 카카오API를 사용하여 주소 -> 좌표 변환
def get_location(address):
  url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
  headers = {"Authorization": "KakaoAK 18778ab7165cea4f52015651131771e1"}
  api_json = json.loads(str(requests.get(url,headers=headers).text))
  address = api_json['documents'][0]['address']
  crd = {"lat": str(address['y']), "lng": str(address['x'])}
  address_name = address['address_name']

  return crd

# 파일에서 주소를 위경도로 변환해서 crd에 넣기
crd = []

for i in range(0, len(goyang['주소']), 1):
    try:
        crd.append(get_location(goyang['주소'][i]))
    except:
        crd.append({'lat':0,'lng':0})

# crd를 데이터프레임화 하여 원래 파일에 붙이기
busan = pd.DataFrame(crd)
goyang = pd.concat([goyang,busan], axis=1)
goyang.to_csv('./data/고양시_위경도.csv')

In [8]:
goyang[goyang['lat']==0]

,Unnamed: 0,운영기관,충전소,충전기ID,충전기타입,지역,시군구,주소,이용자 제한,주소2,lat,lng
93,1058,한국전력,식사동 일산자이2단지 아파트,1,DC차데모+AC3상+DC콤보,경기도,고양시,경기도 고양시 일산동구 식사동 350,시설 상황에 따라 이용이 제한될 수 있음,산동구 식사동 350,0,0
94,1059,한국전력,식사동 일산자이2단지 아파트,2,DC차데모+AC3상+DC콤보,경기도,고양시,경기도 고양시 일산동구 식사동 350,시설 상황에 따라 이용이 제한될 수 있음,산동구 식사동 350,0,0
95,1060,한국전력,식사동 일산자이2단지 아파트,3,AC완속,경기도,고양시,경기도 고양시 일산동구 식사동 350,시설 상황에 따라 이용이 제한될 수 있음,산동구 식사동 350,0,0
96,1061,한국전력,식사동 일산자이2단지 아파트,4,AC완속,경기도,고양시,경기도 고양시 일산동구 식사동 350,시설 상황에 따라 이용이 제한될 수 있음,산동구 식사동 350,0,0
97,1062,한국전력,식사동 일산자이2단지 아파트,5,AC완속,경기도,고양시,경기도 고양시 일산동구 식사동 350,시설 상황에 따라 이용이 제한될 수 있음,산동구 식사동 350,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
934,13527,파워큐브,탄현5단지건영,3,AC완속,경기도,고양시,경기도 고양시 일산서구 현충로 61,거주자외 출입제한,산서구 현충로 61,0,0
935,13528,파워큐브,탄현5단지건영,4,AC완속,경기도,고양시,경기도 고양시 일산서구 현충로 61,거주자외 출입제한,산서구 현충로 61,0,0
936,13529,파워큐브,탄현5단지건영,5,AC완속,경기도,고양시,경기도 고양시 일산서구 현충로 61,거주자외 출입제한,산서구 현충로 61,0,0
1115,15523,스타코프,더휴 빌라 (저압수용가),1,AC완속,경기도,고양시,경기도 고양시 덕양구 서오릉로532번길 92경기 고양시 덕양구 서오릉로532번길 92,해당 시설 정책에 따라 이용이 불가할 수 있습니다.,양구 서오릉로532번길 92경기 고양시 덕양구 서오릉로532번길 92(용두동 462-3),0,0
